# Visibility analysis

The visibility analysis will, at first, be conducted using the "Arealstatistik" (https://www.bfs.admin.ch/bfs/de/home/dienstleistungen/geostat/geodaten-bundesstatistik/boden-nutzung-bedeckung-eignung/arealstatistik-schweiz.assetdetail.25885691.html).

At a later stage it might be conducted using a DEM (https://www.swisstopo.admin.ch/de/hoehenmodell-dhm25).

In [1]:
# setup
from geospatial import *
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import getpass

In [2]:
user_name = getpass.getuser()

# create a path to the CSV file
csv_file_path = os.path.join('/Users', user_name, 'Documents', 'GitHub', 'G877_Alivand', 'arealstatistik_Grindelwald.csv')

# load land use statistic data
arealstatistik_GW = pd.read_csv(csv_file_path, sep=';')
#print(arealstatistik_GW)
#arealstatistik_GW.info()

In [3]:
# create a path to the GeoJSON file of the roads
root = os.path.join('/Users', user_name, 'Documents', 'GitHub', 'G877_Alivand')
fn = 'Roads_small.geojson'
full_path = os.path.join(root, fn)
# load data
roads = gpd.read_file(full_path)
#print(roads)
#roads.info()

In [4]:
# create a path to the file of the DEM
dhm25_file_path = os.path.join('/Users', user_name, 'Documents', 'GitHub', 'G877_Alivand', 'DHM25_Grindelwald.geojson')
# load data
dhm25 = gpd.read_file(dhm25_file_path)
#print(dhm25)

## Setting up the classes

In [5]:
# add the data into the Point class and create an attribute containing the land use category
points_LUS = []

for x in range(len(arealstatistik_GW)):
    a = (Point(arealstatistik_GW['E_COORD'].iloc[x], arealstatistik_GW['N_COORD'].iloc[x]))
    a.cat = arealstatistik_GW['AS18_4'].iloc[x]
    points_LUS.append(a)

print(f'The first 5 points are: {points_LUS[:5]}')
print(f'There are {len(points_LUS)} points in the dataset.')

The first 5 points are: [Point(x=2620000, y=1150100), Point(x=2620000, y=1151400), Point(x=2620000, y=1151800), Point(x=2620000, y=1151900), Point(x=2620000, y=1152200)]
There are 120701 points in the dataset.


In [6]:
# split up the DHM25 geometry to fit in our own Point class
dhm25['E_COORD'] = dhm25['geometry'].x.round(1)
dhm25['N_COORD'] = dhm25['geometry'].y.round(1)
dhm25['Z_COORD'] = dhm25['geometry'].z.round(0).astype(int)
#print(dhm25)

In [7]:
# create points from this data
points_DHM25 = []

for i in range(len(dhm25)):
    point = Point(dhm25['E_COORD'].iloc[i], dhm25['N_COORD'].iloc[i])
    point.z = dhm25['Z_COORD'].iloc[i]  # Append 'Z_COORD' attribute to the point
    points_DHM25.append(point)

print(f'The first 5 points are: {points_DHM25[:5]}')
print(f'There are {len(points_DHM25)} points in the dataset.')
print(f'The elevation of the first point is {points_DHM25[0].z}m.a.s.l.')

The first 5 points are: [Point(x=2625485.9, y=1172295.3), Point(x=2627831.3, y=1172885.9), Point(x=2639148.4, y=1170993.8), Point(x=2648728.1, y=1170739.1), Point(x=2652621.9, y=1170375.0)]
There are 4280 points in the dataset.
The elevation of the first point is 1584m.a.s.l.


### Main (land use statistic)

Here the visibility analysis based on the land use statistic (Arealstatistik) is run.

In [8]:
closest_pixel_centers = []

for point in points_LUS[:10]:
    print(point)
    closest_pixel_center = point.find_closest_pixel(points_DHM25)
    print("Closest pixel center:", closest_pixel_center)
    closest_pixel_centers.append(closest_pixel_center)

# closest_pixel_centers now contains the closest pixel centers for each Point instance in points_LUS

Point(x=2620000, y=1150100)
Closest pixel center: Point(x=2620546.9, y=1150315.6)
Point(x=2620000, y=1151400)
Closest pixel center: Point(x=2620100.0, y=1151614.1)
Point(x=2620000, y=1151800)
Closest pixel center: Point(x=2620100.0, y=1151614.1)
Point(x=2620000, y=1151900)
Closest pixel center: Point(x=2620118.8, y=1152110.9)
Point(x=2620000, y=1152200)
Closest pixel center: Point(x=2620118.8, y=1152110.9)
Point(x=2620000, y=1152400)
Closest pixel center: Point(x=2620118.8, y=1152110.9)
Point(x=2620000, y=1152800)
Closest pixel center: Point(x=2620120.3, y=1152823.4)
Point(x=2620000, y=1153200)
Closest pixel center: Point(x=2620034.4, y=1153428.1)
Point(x=2620000, y=1153500)
Closest pixel center: Point(x=2620034.4, y=1153428.1)
Point(x=2620000, y=1154000)
Closest pixel center: Point(x=2620064.0, y=1154006.3)


In [9]:
# find closest pixel center for a given coordinate
def find_closest_pixel(x_coord, y_coord):
    pixel_centers = arealstatistik_GW[['E_COORD', 'N_COORD']].values
    
    distances = np.sqrt((pixel_centers[:, 0] - x_coord)**2 + (pixel_centers[:, 1] - y_coord)**2)
    
    closest_index = np.argmin(distances)
    
    closest_pixel_center = pixel_centers[closest_index]
    
    return closest_pixel_center

# test
x_coord = 2625321
y_coord = 1162324
closest_pixel_center = find_closest_pixel(x_coord, y_coord)
print("Closest pixel center:", closest_pixel_center)

Closest pixel center: [2625300 1162300]


In [10]:
# find neighbor of a given pixel center
def find_neighbors(center_pixel):
    x, y = center_pixel
    neighbors = []
    for dx in [-100, 0, 100]:
        for dy in [-100, 0, 100]:
            neighbors.append((x + dx, y + dy))
    return neighbors

# test
neighborhood = find_neighbors(closest_pixel_center)
print(neighborhood)

[(2625200, 1162200), (2625200, 1162300), (2625200, 1162400), (2625300, 1162200), (2625300, 1162300), (2625300, 1162400), (2625400, 1162200), (2625400, 1162300), (2625400, 1162400)]


In [11]:
# get landcover values for neighborhood
def get_values_for_pixels(pixel_centers):
    values = arealstatistik_GW.loc[(arealstatistik_GW['E_COORD'].isin([x for x, _ in pixel_centers])) & 
                                   (arealstatistik_GW['N_COORD'].isin([y for _, y in pixel_centers])), 'AS18_4']
    return values

# test
arealstatistik_values = get_values_for_pixels(neighborhood)
print(arealstatistik_values)

1236      3
62379     2
71885     2
82629     4
106077    3
110461    3
111485    4
113468    3
120482    4
Name: AS18_4, dtype: int64


In [12]:
# determine scenicness
def is_neighborhood_scenic(arealstatistik_values):
    count_1_3 = sum(1 for value in arealstatistik_values if value in [1, 3])

    if count_1_3 > 4:
        return False
    
    return True

# test
scenicness_test = is_neighborhood_scenic(arealstatistik_values)
print(scenicness_test)

True


In [13]:
# determine scenicness
def neighborhood_scenicness(neighborhood):
    count = sum(1 for value in arealstatistik_values if value in [1, 3])
    score = count / 9
    return score

# test
scenicness_score = neighborhood_scenicness(neighborhood)
print(scenicness_score)

0.4444444444444444


## Main (viewshed)

Here the visibility analysis based on the DEM is run.